In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dependencies

In [ ]:
!pip install llama-index pypdf
!pip install llama-index-embeddings-huggingface
!pip install -U llama_hub llama_index braintrust autoevals pypdf pillow transformers torch torchvision

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "" #your private api key

# Basic RAG Review

## Using llama index a dataframework for LLM's

In [ ]:
!pip install llama-index-llms-huggingface

In [ ]:
from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex, ServiceContext
# from llama_index.llms import HuggingFaceLLM
from llama_index.llms.openai import OpenAI
import json

### Loading Document ( **PHYSIO THERAPY TREATMENTS pdf** in this case)

In [ ]:
loader = PDFReader()
docs0 = loader.load_data(file=Path("/content/drive/MyDrive/LLM/LG-physio-treatments.pdf"))

In [ ]:
docs0[30]

Document(id_='042d2f38-8469-43df-b640-cf55d110009d', embedding=None, metadata={'page_label': '31', 'file_name': '/content/drive/MyDrive/LLM/LG-physio-treatments.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text=' \n \n \nPhysiotherapy Learner Guide: Assist with physiotherapy treatments and interventions  - 27 - \n Confidentiality  \nQueensland Health has a commitment to ensuring the privacy and confidentiality of all \npersonal info rmation collected. Patient information is confidential and care should be \ntaken to ensure that all documented information remains confidential.  \n \nListed below are some general guidelines for maintaining patient confidentiality:  \n\uf0b7 do not allow anyone to touc h or look at a patient record unless they are a \nhealthcare provider taking care of that patient  \n\uf0b7 carry medical records in a way so as not to expose identifying information \nsuch as patient details  \n\uf0b7 keep all patient records in 

In [ ]:
from llama_index.core import Document
doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

In [ ]:
### Step 2: Parsing Documents into Text Chunks (Nodes)

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode

In [ ]:
node_parser = SentenceSplitter(chunk_size=1024)

In [ ]:
base_nodes = node_parser.get_nodes_from_documents(docs)
base_nodes[0]

TextNode(id_='d9f3b1fd-e5e7-44a7-ae56-965f7bc286ce', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3a2f813d-f3ec-49cc-93b2-93d15648e308', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='d4d87ca3e5c1e5a723f4782e4edb206c1cc87bd43b41d7f5a7b5efa7b3c58292'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='ae0dd353-46eb-4f3e-afbe-1165b24aedeb', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='4a5f73321dcf7be455cd775106805b2c47b5aa3c3225faf0385d46fb6c025d86')}, text='Allied Health Professions’ Office of Queensland  \nPhysiotherapy Learner Guide  \n \nAssist with physiotherapy treatments and \ninterventions  \nApril  2017\n \n\n \n \n \nPhysiotherapy Learner Guide: Assist with physiotherapy treatments and interventions  - ii - \n Physiotherapy Learner Guide – Assist with physiotherapy treatments and interventions  \nPublished by the State of Queensland (Que

In [ ]:
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [ ]:
base_nodes[22]

TextNode(id_='node-22', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3a2f813d-f3ec-49cc-93b2-93d15648e308', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='d4d87ca3e5c1e5a723f4782e4edb206c1cc87bd43b41d7f5a7b5efa7b3c58292'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='3068bd5e-15e1-4da6-8ebf-e11d965cf131', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='308b55c60a0a0ae952a3d2cbd271b2435263d171c59f3f811ebba2d24a52c0ec'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='d555d9dc-000f-4d61-a83c-b1b96be0b2b4', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='756b23bb571345b8f5474094a06196e59f3a79a3b00a4e9ec6e12ba61882f8e2')}, text='For example, a patient may develop a tolerance \nfor their pain medication, requiring increased doses and multiple medications, which \ncan lead to sleep distu rbances , weight gain and depression.   \n\n \

### choosing embedding model and llm model from open ai

In [ ]:
from llama_index.core.embeddings import resolve_embed_model

embed_model = resolve_embed_model("local:BAAI/bge-small-en")
llm = OpenAI(model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

<ipython-input-36-0230228534f0>:5: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


### creating basic index. retriever and query engine

In [ ]:
base_index = VectorStoreIndex(base_nodes, service_context=service_context)
base_retriever = base_index.as_retriever(similarity_top_k=2)

In [ ]:
retrievals = base_retriever.retrieve(
    "suggest me a physio therapy treatments for knee injury"
)

In [ ]:
for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-26<br>**Similarity:** 0.8713409657428597<br>**Text:** Physiotherapy treatment programs are very 
effective in improving  and maintaining a patient’s mobility and this can h ave important 
flow-on effects by making sure their lungs are also ‘exercised’, thus helping to prevent 
lung collapse and aid the removal of secretions.  
 
Exercising for Mobility is covered in detail in the unit HLTAH403B Deliver and monitor 
exercise progra m for mobility , but here is a quick overview.  
 
You may be asked to complete a mobility program including:  
 walking  
 sit to stand  
 balance t raining  
 stair climbing, etc  
 
Some of the equipment you may use during mobility treatment sessions includes:  
 walking aid s (rollator frames, four wheel walkers, pick -up frames, walking sticks, 
axillary  crutches, forearm crutches  and four point sticks).  
 walk belts  
 hoists (sling hoists, sit to stand hoists  and walking hoists)  
 wheelchairs  
 slide boards  
 parallel bars  

 
 
 
Physiotherapy Learner Guide: Assist with physiotherapy treatments and interventions  - 65 - 
  
You will n eed to monitor and report the following to your physiotherapist, and/or record 
in the medical chart:  
 distance mobilised  
 exercises performed  
 aids used  
 reports from patient (e.g. fatigue, pain)  
 adverse events (e.g. falls, pain  etc). These adverse reactions n eed to be 
reported promptly to your physiotherapist , nurse or medical team  
 
Mobilising a patient works on their respiratory system like this:  
 muscles require m...<br>

**Node ID:** node-28<br>**Similarity:** 0.8596733466554606<br>**Text:** May also be able to assist clien ts in doing exercises in an active -
assisted manner e.g. straight leg raise (SLR)  
Rowing machine  Can be used to improve endurance and cardiovascular fitness  
Stair climber or stepper  Can be used to increase cardiovascular fitness and endurance  
Standing f rames  Standing frames can be used as part of a postural management  

 
 
 
Physiotherapy Learner Guide: Assist with physiotherapy treatments and interventions  - 70 - 
   program, and can assist to increase independence, mobility, and 
self-esteem. Standers may be used by people with mild to severe 
impairments, including spinal cord injury, traumatic brain injury , 
cerebral palsy , spina bifida , muscular dystrophy , multiple 
sclerosis  and stroke . 
Exercise balls  Can be used as part of strengthening programs or balance and 
stability training  
Resistance bands  Can be used for strengthening muscles isometrically and 
isotonically  
Tilt tables  Can be used with clients who have had periods of bed rest and 
are too weak to stand, or to reintroduce to a vertical position. Tilt 
tabling can be used to facilitate weight bearing, prevent muscle 
contractures, improve lower limb strength and inc rease arousal.  
Treadmills  Can be used for exercise testing, to improve endurance and 
cardiovascular fitness, and in gait retraining  
Incentive spirometer  Can be used with clients post -operatively or in appropriate clients 
with poor respiratory effort or s igns of collapse in the l...<br>

In [ ]:
query_engine_base = RetrieverQueryEngine.from_args(base_retriever, llm=llm)
response = query_engine_base.query(
    "suggest me a physio therapy treatments for knee injury"
)
print(str(response))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

# Installing TensorRT

In [ ]:
!pip install tensorrt_llm --extra-index-url https://pypi.nvidia.com --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com, https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 893.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.9/327.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (set

In [ ]:
!git clone https://github.com/NVIDIA/TensorRT-LLM.git
!cd TensorRT-LLM
!git submodule update --init --recursive

Cloning into 'TensorRT-LLM'...
remote: Enumerating objects: 10492, done.
remote: Counting objects: 100% (452/452), done.
remote: Compressing objects: 100% (318/318), done.
remote: Total 10492 (delta 128), reused 387 (delta 111), pack-reused 10040
Receiving objects: 100% (10492/10492), 131.26 MiB | 20.43 MiB/s, done.
Resolving deltas: 100% (7109/7109), done.
Updating files: 100% (1950/1950), done.
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!git clone https://github.com/NVIDIA/trt-llm-rag-windows.git

Cloning into 'trt-llm-rag-windows'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 138 (delta 6), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (138/138), 1.34 MiB | 4.95 MiB/s, done.
Resolving deltas: 100% (60/60), done.
